# **Optimal meal selection at McDonalds.**

In [ ]:
import numpy as np
import pandas as pd
!pip install pulp 
from pulp import *
import os
import pulp as p 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Datasets:

In [ ]:
Mcdata = pd.read_excel('/content/drive/MyDrive/Datasets/Food_data.xlsx')
food = pd.read_excel('/content/drive/MyDrive/Datasets/Nutrional_requirements.xlsx')

In [ ]:
Mcdata_1=Mcdata.set_index('Name')
Mcdata_1.head(2)

,cost,Cal,CalFat,Fat,SatFat,Chol,Sodium,Carbo,Protein,VitA,VitC,Calcium,Iron
Name,,,,,,,,,,,,,
Hamburger,0.59,255,80,9.0,3.0,35,490,30.0,12,4,4,10,15
Cheeseburger,0.69,305,120,13.0,5.0,50,725,30.0,15,8,4,20,15


In [ ]:
food_1=food.T
food_1=food_1.rename(columns={0:'Cal',1:'CalFat',2:'Fat',3:'SatFat',4:'Chol',5:'Sodium',6:'Carbo',7:'Protein',8:'VitA',9:'VitC',10:'Calcium',11:'Iron'}).drop(['Name'],axis=0)
food_1

,Cal,CalFat,Fat,SatFat,Chol,Sodium,Carbo,Protein,VitA,VitC,Calcium,Iron
UOM,calories,calories,grams,grams,milligrams,milligrams,grams,grams,percent,percent,percent,percent
Min_RDA,2000,NaN,NaN,NaN,NaN,NaN,350,55,100,100,100,100
Max_RDA,NaN,NaN,100,30,375,3000,375,NaN,NaN,NaN,NaN,NaN


Datasets were concated for generalization.

In [ ]:
df = pd.concat([food_1,Mcdata_1])
df.head(5)

,Cal,CalFat,Fat,SatFat,Chol,Sodium,Carbo,Protein,VitA,VitC,Calcium,Iron,cost
UOM,calories,calories,grams,grams,milligrams,milligrams,grams,grams,percent,percent,percent,percent,NaN
Min_RDA,2000,NaN,NaN,NaN,NaN,NaN,350,55,100,100,100,100,NaN
Max_RDA,NaN,NaN,100,30,375,3000,375,NaN,NaN,NaN,NaN,NaN,NaN
Hamburger,255,80,9,3,35,490,30,12,4,4,10,15,0.59
Cheeseburger,305,120,13,5,50,725,30,15,8,4,20,15,0.69


In [ ]:
# Convert the item names to a list
MenuItems = Mcdata.Name.tolist()
MenuItems

['Hamburger',
 'Cheeseburger',
 'Quarter Pound Cheeseburger',
 'Local Minimum',
 'Local Minimum with Cheese',
 'Global Maximum',
 'Fish Filet Sandwich',
 'Grilled Chicken Sandwich',
 'Fried Chicken Sandwich',
 'Fries, small',
 'Fries, large',
 'Fries, optimal',
 'Chicken Nuggets (6 pcs)',
 'Chicken Nuggets (9 pcs)',
 'Chicken Nuggets (20 pcs)',
 'Hot Mustard Sauce',
 'Barbeque Sauce',
 'Sweet & Sour Sauce',
 'Honey',
 'Chef Salad',
 'Fried Chicken Salad',
 'Garden Salad',
 'Side Salad',
 'Croutons',
 'Bacon Bits',
 'Bleu Cheese Dressing',
 'Ranch Dressing',
 '1000 Island Dressing',
 'Lite Vinaigrette Dressing',
 'French Reduced Cal Dressing',
 'Toasted with Egg',
 'Toasted with Sausage',
 'Toasted with Sausage & Egg',
 'Toasted English Muffin',
 'Sausage Biscuit',
 'Sausage Biscuit with Egg',
 'Bacon, Egg & Cheese Biscuit',
 'Hash Browns',
 'Breakfast Burrito',
 'Cheerios',
 'Wheaties',
 'Apple Danish',
 'Cheese Danish',
 'Cinnamon Raisin Danish',
 'Raspberry Danish',
 'Lowfat Frozen Y

In [ ]:
MenuItems_vars = p.LpVariable.dicts("MenuItems",MenuItems,lowBound=0,
   upBound=10,cat='Integer')

In [ ]:
# Convert all of the macro nutrients fields to be dictionaries of the item names
cost = Mcdata.set_index('Name')['cost'].to_dict()
Cal = Mcdata.set_index('Name')['Cal'].to_dict()
CalFat =  Mcdata.set_index('Name')['CalFat'].to_dict()
Fat =  Mcdata.set_index('Name')['Fat'].to_dict()
SatFat =  Mcdata.set_index('Name')['SatFat'].to_dict()
Chol = Mcdata.set_index('Name')['Chol'].to_dict()
Sodium =  Mcdata.set_index('Name')['Sodium'].to_dict()
Carbo =  Mcdata.set_index('Name')['Carbo'].to_dict()
Protein = Mcdata.set_index('Name')['Protein'].to_dict()
VitA =  Mcdata.set_index('Name')['VitA'].to_dict()
VitC = Mcdata.set_index('Name')['VitC'].to_dict()
Calcium =  Mcdata.set_index('Name')['Calcium'].to_dict()
Iron =  Mcdata.set_index('Name')['Iron'].to_dict()

In [ ]:
# Set it up as a minimization problem
prob = p.LpProblem("McOptimization_Problem", p.LpMinimize)

In [ ]:
# First entry is the cost calculation (this is our objective)
prob += p.lpSum([cost[i]*MenuItems_vars[i] for i in MenuItems]),"cost"

In [ ]:
Sauces=['Barbeque Sauce', "Sweet & Sour Sauce" ,'Hot Mustard Sauce','Honey']
Drinks=['Vanilla Shake' ,'Chocolate Shake',
     'Strawberry Shake' ,'1% Lowfat Milk', 'Orange Juice',
     'Coca-Cola (small)' ,'Coca-Cola (medium)', 'Coca-Cola (large)' ,
     'Diet Coke (small)' ,'Diet Coke (medium)', 'Diet Coke (large)', 
     'Sprite (small)', 'Sprite (medium)', 'Sprite (large)',
     'H-C Orange Drink (small)', 'H-C Orange Drink (medium)', 
     'H-C Orange Drink (large)']

In [ ]:
#Sauces were limited to maximum two packs per day
prob+=p.lpSum([MenuItems_vars[i] for i in Sauces])<=2,'Saucelimit'

In [ ]:
#Drinks were limited to maximum three per day
prob+=p.lpSum([MenuItems_vars[i] for i in Drinks])<=3,'Drinklimit'

In [ ]:
#cal must be >= 2000 calories
prob += p.lpSum([Cal[i]*MenuItems_vars[i] for i in MenuItems]) >=2000, "Cal"

In [ ]:
# Total Fat must be <= 100 g
prob += p.lpSum([Fat[i]*MenuItems_vars[i] for i in MenuItems]) <=100, "Fat"

In [ ]:
# Saturated Fat must be <= 30 g
prob += p.lpSum([SatFat[i]*MenuItems_vars[i] for i in MenuItems]) <= 30, "Saturated Fat"

In [ ]:
#Cholestrol must be <= 375 mg
prob += p.lpSum([Chol[i]*MenuItems_vars[i] for i in MenuItems]) <= 375, "Cholestrol"

In [ ]:
# Carbohydrates must be between 350-375g g
prob += p.lpSum([Carbo[i]*MenuItems_vars[i] for i in MenuItems]) >= 350, "Carbo_lower"
prob += p.lpSum([Carbo[i]*MenuItems_vars[i] for i in MenuItems]) <= 375, "Carbo_upper"

In [ ]:
# Protein must be >= 55 g
prob += p.lpSum([Protein[i]*MenuItems_vars[i] for i in MenuItems]) >= 55, "Protein"

In [ ]:
# Sodium <= 6000 mg
prob += p.lpSum([Sodium[i]*MenuItems_vars[i] for i in MenuItems]) <= 3000, "Sodium"

In [ ]:
#VitA must be >= 100 percent
prob += p.lpSum([VitA[i]*MenuItems_vars[i] for i in MenuItems]) >= 100, "Vitamin A"

In [ ]:
#VitC must be >= 100 percent
prob += p.lpSum([VitC[i]*MenuItems_vars[i] for i in MenuItems]) >= 100, "Vitamin C"

In [ ]:
#Calcuim must be >= 100 percent
prob += p.lpSum([Calcium[i]*MenuItems_vars[i] for i in MenuItems]) >= 100, "Calcuim"

In [ ]:
#iron must be >= 100 percent
prob += p.lpSum([Iron[i]*MenuItems_vars[i] for i in MenuItems]) >= 100, "Iron"

In [ ]:
print(prob)

McOptimization_Problem:
MINIMIZE
0.6*MenuItems_1%_Lowfat_Milk + 0.84*MenuItems_Apple_Danish + 1.49*MenuItems_Bacon,_Egg_&_Cheese_Biscuit + 0.99*MenuItems_Breakfast_Burrito + 1.09*MenuItems_Cheerios + 0.84*MenuItems_Cheese_Danish + 0.69*MenuItems_Cheeseburger + 2.94*MenuItems_Chef_Salad + 4.49*MenuItems_Chicken_Nuggets_(20_pcs) + 1.69*MenuItems_Chicken_Nuggets_(6_pcs) + 2.34*MenuItems_Chicken_Nuggets_(9_pcs) + 1.09*MenuItems_Chocolate_Shake + 0.89*MenuItems_Cinnamon_Raisin_Danish + 1.17*MenuItems_Coca_Cola_(large) + 0.99*MenuItems_Coca_Cola_(medium) + 0.88*MenuItems_Coca_Cola_(small) + 1.17*MenuItems_Diet_Coke_(large) + 0.99*MenuItems_Diet_Coke_(medium) + 0.88*MenuItems_Diet_Coke_(small) + 1.44*MenuItems_Fish_Filet_Sandwich + 2.99*MenuItems_Fried_Chicken_Salad + 2.04*MenuItems_Fried_Chicken_Sandwich + 1.17*MenuItems_Fries,_large + 1.49*MenuItems_Fries,_optimal + 0.77*MenuItems_Fries,_small + 1.99*MenuItems_Garden_Salad + 1.84*MenuItems_Global_Maximum + 2.29*MenuItems_Grilled_Chicken_San

In [ ]:
solution=prob.solve()

In [ ]:
print("Status:", p.LpStatus[prob.status])

Status: Optimal


In [ ]:
# Get the total cost (minimized)
print("Total cost = ", value(prob.objective))

Total cost =  8.040000000000001


In [ ]:
output={}
for v in prob.variables():
  if v.varValue!=0:
    output[v.name] = int(v.varValue)

In [ ]:
output

{'MenuItems_Cheerios': 1,
 'MenuItems_Chocolate_Shake': 2,
 'MenuItems_Cinnamon_Raisin_Danish': 1,
 'MenuItems_Croutons': 3,
 'MenuItems_Hamburger': 3,
 'MenuItems_Honey': 2,
 'MenuItems_Orange_Juice': 1,
 'MenuItems_Side_Salad': 1}

In [ ]:
# Loop over the constraint set and get the final solution
results = {}
for constraint in prob.constraints:
    s = 0
    for var, coefficient in prob.constraints[constraint].items():
        s += var.varValue*coefficient
        results[prob.constraints[constraint].name.replace('_lower','').replace('_upper','')] = int(s)

In [ ]:
results

{'Cal': 2325,
 'Calcuim': 114,
 'Carbo': 352,
 'Cholestrol': 235,
 'Drinklimit': 3,
 'Fat': 69,
 'Iron': 101,
 'Protein': 76,
 'Saturated_Fat': 26,
 'Saucelimit': 2,
 'Sodium': 2915,
 'Vitamin_A': 125,
 'Vitamin_C': 175}